# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 5 2023 8:30AM,254308,10,SOTAH0000878,"Nextsource Biotechnology, LLC",Released
1,Jan 5 2023 8:30AM,254301,10,8990139-SD,Eywa Pharma Inc.,Released
2,Jan 5 2023 8:30AM,254306,10,8990148-SD,Eywa Pharma Inc.,Released
3,Jan 4 2023 4:11PM,254310,10,8990153-SD,Eywa Pharma Inc.,Released
4,Jan 4 2023 3:56PM,254307,19,ADV80010046,"AdvaGen Pharma, Ltd",Released
5,Jan 4 2023 3:56PM,254307,19,ADV80010048,"AdvaGen Pharma, Ltd",Released
6,Jan 4 2023 3:56PM,254307,19,ADV80010049,"AdvaGen Pharma, Ltd",Released
7,Jan 4 2023 3:56PM,254307,19,ADV80010050,"AdvaGen Pharma, Ltd",Released
8,Jan 4 2023 3:56PM,254307,19,ADV80010051,"AdvaGen Pharma, Ltd",Released
9,Jan 4 2023 3:56PM,254307,19,ADV80010054,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,254305,Released,1
19,254306,Released,1
20,254307,Released,6
21,254308,Released,1
22,254310,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254305,NaN,NaN,1.0
254306,NaN,NaN,1.0
254307,NaN,NaN,6.0
254308,NaN,NaN,1.0
254310,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254196,0.0,0.0,17.0
254230,38.0,3.0,58.0
254249,0.0,1.0,0.0
254255,0.0,1.0,0.0
254265,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254196,0,0,17
254230,38,3,58
254249,0,1,0
254255,0,1,0
254265,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254196,0,0,17
1,254230,38,3,58
2,254249,0,1,0
3,254255,0,1,0
4,254265,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254196,,,17
1,254230,38,3,58
2,254249,,1,
3,254255,,1,
4,254265,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 5 2023 8:30AM,254308,10,"Nextsource Biotechnology, LLC"
1,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.
2,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.
3,Jan 4 2023 4:11PM,254310,10,Eywa Pharma Inc.
4,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd"
10,Jan 4 2023 3:52PM,254305,19,"Emersa Waterbox, LLC"
11,Jan 4 2023 3:48PM,254304,19,"GUSA Granules USA, Inc."
12,Jan 4 2023 3:39PM,254303,12,SugarBear
13,Jan 4 2023 3:36PM,254302,15,"Virtus Pharmaceuticals, LLC"
15,Jan 4 2023 3:30PM,254297,19,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 5 2023 8:30AM,254308,10,"Nextsource Biotechnology, LLC",,,1
1,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,,,1
2,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,,,1
3,Jan 4 2023 4:11PM,254310,10,Eywa Pharma Inc.,,,1
4,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd",,,6
5,Jan 4 2023 3:52PM,254305,19,"Emersa Waterbox, LLC",,,1
6,Jan 4 2023 3:48PM,254304,19,"GUSA Granules USA, Inc.",,,1
7,Jan 4 2023 3:39PM,254303,12,SugarBear,,,1
8,Jan 4 2023 3:36PM,254302,15,"Virtus Pharmaceuticals, LLC",,,2
9,Jan 4 2023 3:30PM,254297,19,"Emersa Waterbox, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 5 2023 8:30AM,254308,10,"Nextsource Biotechnology, LLC",1,,
1,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1,,
2,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,1,,
3,Jan 4 2023 4:11PM,254310,10,Eywa Pharma Inc.,1,,
4,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd",6,,
5,Jan 4 2023 3:52PM,254305,19,"Emersa Waterbox, LLC",1,,
6,Jan 4 2023 3:48PM,254304,19,"GUSA Granules USA, Inc.",1,,
7,Jan 4 2023 3:39PM,254303,12,SugarBear,1,,
8,Jan 4 2023 3:36PM,254302,15,"Virtus Pharmaceuticals, LLC",2,,
9,Jan 4 2023 3:30PM,254297,19,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 5 2023 8:30AM,254308,10,"Nextsource Biotechnology, LLC",1,,
1,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1,,
2,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,1,,
3,Jan 4 2023 4:11PM,254310,10,Eywa Pharma Inc.,1,,
4,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd",6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 5 2023 8:30AM,254308,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1.0,NaN,NaN
2,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,1.0,NaN,NaN
3,Jan 4 2023 4:11PM,254310,10,Eywa Pharma Inc.,1.0,NaN,NaN
4,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd",6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 5 2023 8:30AM,254308,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Jan 5 2023 8:30AM,254301,10,Eywa Pharma Inc.,1.0,0.0,0.0
2,Jan 5 2023 8:30AM,254306,10,Eywa Pharma Inc.,1.0,0.0,0.0
3,Jan 4 2023 4:11PM,254310,10,Eywa Pharma Inc.,1.0,0.0,0.0
4,Jan 4 2023 3:56PM,254307,19,"AdvaGen Pharma, Ltd",6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1779939,23.0,10.0,0.0
12,254303,1.0,0.0,0.0
15,762802,61.0,3.0,38.0
16,254265,1.0,0.0,0.0
19,1525750,23.0,1.0,0.0
21,254266,1.0,0.0,0.0
22,254291,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1779939,23.0,10.0,0.0
1,12,254303,1.0,0.0,0.0
2,15,762802,61.0,3.0,38.0
3,16,254265,1.0,0.0,0.0
4,19,1525750,23.0,1.0,0.0
5,21,254266,1.0,0.0,0.0
6,22,254291,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,23.0,10.0,0.0
1,12,1.0,0.0,0.0
2,15,61.0,3.0,38.0
3,16,1.0,0.0,0.0
4,19,23.0,1.0,0.0
5,21,1.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,23.0
1,12,Released,1.0
2,15,Released,61.0
3,16,Released,1.0
4,19,Released,23.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,0.0,0.0,38.0,0.0,0.0,0.0,0.0
Executing,10.0,0.0,3.0,0.0,1.0,0.0,0.0
Released,23.0,1.0,61.0,1.0,23.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,38.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,3.0,0.0,1.0,0.0,0.0
2,Released,23.0,1.0,61.0,1.0,23.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,38.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,3.0,0.0,1.0,0.0,0.0
2,Released,23.0,1.0,61.0,1.0,23.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()